# Machine learning with `nilearn`

Although nilearn's visualizations are quite nice, its primary purpose was to facilitate machine learning in neuroimaging. It's in some sense the bridge between [nibabel](http://nipy.org/nibabel/) and [scikit-learn](http://scikit-learn.org/stable/). On the one hand, it reformats images to be easily passed to scikit-learn, and on the other, it reformats the results to produce valid nibabel images.

So let's take a look at a short MVPA example.

**Note**: This section is heavily based on the [nilearn decoding tutorial](https://nilearn.github.io/auto_examples/plot_decoding_tutorial.html).

# Setup

In [ ]:
%matplotlib inline
from nilearn import datasets, plotting, input_data, image
import numpy as np

# Load example dataset

Before we can do anything, we first need to download the example dataset. The whole dataset is almost 300MB big, and therefore might take some time to download.

In [ ]:
haxby_dataset = datasets.fetch_haxby(data_dir='data')

bold = haxby_dataset.func[0]
mask = haxby_dataset.mask_vt[0]
anat = haxby_dataset.anat[0]
labels = haxby_dataset.session_target[0]

!nib-ls $bold

# Masking and Un-masking data

We need our functional data in a 2D, sample-by-voxel matrix. To get that, we'll select a set of voxels in VT cortex defined by `mask`.

In [ ]:
plotting.plot_roi(mask, anat, cmap='Paired', dim=-.5)

`NiftiMasker` is an object that applies a mask to a dataset and returns the masked voxels as a vector at each time point.
`standardize=True` z-scores each voxel.

In [ ]:
masker = input_data.NiftiMasker(mask_img=mask, standardize=True)
samples = masker.fit_transform(bold)
print(samples)

Its shape corresponds to the number of time-points times the number of voxels in the mask.

In [ ]:
print(samples.shape)

To recover the original data shape (giving us a masked and z-scored BOLD series), we simply use the masker's inverse transform:

In [ ]:
masked_epi = masker.inverse_transform(samples)

Let's now visualize the masked epi.

In [ ]:
max_zscores = image.math_img("np.abs(img).max(axis=3)", img=masked_epi)
plotting.plot_stat_map(max_zscores, bg_img=anat, dim=-.5)

# Simple MVPA Example

Multi-voxel pattern analysis (MVPA) is a general term for techniques that contrast conditions over multiple voxels. It's very common to use machine learning models to generate statistics of interest.

In this case, we'll use the response patterns of voxels in VT cortex to predict the identity of the stimulus this subject was presented with. We'll use a support vector classifier (SVC) and leave-one-run-out cross-validation.

**Note:** This section is not intended to teach machine learning, but to demonstrate a simple nilearn pipeline.

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

The labels file contains metadata for each volume, indicating the stimulus type and run number.

In [ ]:
!head -n 10 $labels

Using `np.recfromcsv()`, we can refer to each column of this file by its header.

In [ ]:
attrs = np.recfromcsv(labels, delimiter=" ")
attrs.shape

In [ ]:
stimuli, runs = attrs['labels'], attrs['chunks']
print(np.unique(stimuli))

In [ ]:
np.unique(runs)

For simplicity, let's consider a two-class problem. Select the BOLD samples associated with bottles and shoes. We'll also need to select the corresponding stimuli and run numbers.

In [ ]:
condition_mask = (stimuli == b'bottle') | (stimuli == b'shoe')

samples_2class = samples[condition_mask]
stimuli_2class = stimuli[condition_mask]
runs_2class = runs[condition_mask]

samples_2class.shape

Leave-one-run-out cross-validation trains on `(n - 1)` runs, and classifies the remaining run, for each run. Mean (across runs) cross-validation accuracy is a common statistic for classification-based MVPA.

In [ ]:
svc = SVC(kernel='linear')
cva = cross_val_score(estimator=svc,
                      X=samples_2class,
                      y=stimuli_2class,
                      groups=runs_2class,
                      cv=LeaveOneGroupOut(),
                      n_jobs=-1)
print(cva, cva.mean(), sep='\n')

Another approach is to train a classifier on all of the data. This isn't useful for predicting, but we can read out the weight assigned to each voxel, giving a measure of its correlation with the stimulus type.

In [ ]:
svc.fit(samples_2class, stimuli_2class)
svc.coef_.shape

Since we have a value for each voxel, we can simply map this back to the volume using our `masker`, and visualize the weights.

In [ ]:
coef_vol = masker.inverse_transform(svc.coef_)
plotting.plot_stat_map(coef_vol, bg_img=anat, dim=-.5)

# Review

In this section, we explored nilearn's tools for interfacing neuroimaging data and machine learning algorithms. Central to this is the concept of the masker, which moves data from 4-dimensional BOLD time series to a 2-dimensional series of feature vectors, and can map resulting statistics back into the original BOLD volume. We used leave-one-run-out cross-validation to explore 2-class support vector classification, and mapped feature weights back into the volume.